# Network Analysis

Many interesting data problems can be fruitfully thought of in terms of networks, consisting of nodes of some type nand the edges that join them.  For instance, your facebook friends form the nodes of a network whose edges are friendship relations.  A less obvious example is the world wide web itself, with each web page a node, and each hyperlink from one page to another an edge.

Facebook friendship is mutual -- If I am facebook friends with you, then necessarily you are friends with me.  In this case, we say that the edges are undirected.  Hyperlinks are not -- my website links to whitehouse.gov, but whitehouse.gov refuses to link to my website.  We call these types of edges directed.  We'll look at both kinds of networks:

In [2]:
import math, random, re
from collections import defaultdict, Counter, deque
from code_python3.linear_algebra import dot, get_row, get_column, make_matrix, magnitude, scalar_multiply, shape, distance
from functools import partial

users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

# give each user a friends list
for user in users:
    user["friends"] = []

# and populate it
for i, j in friendships:
    # this works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i

We were dissatisfied with our notion of degree centrality, which didn't really agree with our intuitions about who were the key connectors of the network.  An alternative metric is betweenness centrality, which identifies people who frequently are on the shortest paths between pairs of other people.  In particular, the betweenness centrality of node `i` is computed by adding up for every other pair of nodes `j` and `k`, the proportion of shortest paths between node j and node k that pass through i.

So, as a first step, we'll need to figure out the shortest paths between all pairs of people.  There are some pretty sophisticated algorithms for doing so efficiently, but we will use a less efficient, easier to understand algorithm.  This algorithm (an implementation of breadth-first search) is a bit more complicated than the others:

1. Our goal is a function that takes a `from_user` and finds all shortest paths to every other user.
2. We'll represent a path as list of user IDs.  Since every path starts at `from_user`, we won't include her ID in the list.  This means that the length of the list representing the path will be the length of the path itself.
3. We'll maintain a dictionary `shortest_paths_to` where the keys are user IDs and the values are lists of paths that end at the user with the specified ID.  If there is a unique shortest path, the list will just contain that one path.  If there are multiple shortest paths, the list will contain all of them.
4. We'll also maintain a queue `frontier` that contains the users we want to explore in the order we want to explore them.  We'll store them as pairs `(prev_user, user)`  so that we know how we got to each one.  We initialize the queue with all the neighbors of `from_user`.
5. As we explore the graph, whenever we find new neighbors that we don't already know shortest paths to, we add them to the end of the queue to explore later, with the current user as `prev_user`.
6. When we take a user off the queue, and we've never encountered that user before, we've definitely found one or more shortest path to him -- each of the shortest paths to `prev_user` with one extra step added.
7. When we take a user off the queue and we have encountered that user before, then either we've found another shortest path or we've found a longer path.
8. When no more users are left on the queue, we've explored the ewhole graph and we're done.

We can put this altogether like:

In [3]:
def shortest_paths_from(from_user):

    # a dictionary from "user_id" to *all* shortest paths to that user
    shortest_paths_to = { from_user["id"] : [[]] }

    # a queue of (previous user, next user) that we need to check.
    # starts out with all pairs (from_user, friend_of_from_user)
    frontier = deque((from_user, friend)
                     for friend in from_user["friends"])

    # keep going until we empty the queue
    while frontier:

        prev_user, user = frontier.popleft() # take from the beginning
        user_id = user["id"]

        # the fact that we're pulling from our queue means that
        # necessarily we already know a shortest path to prev_user
        paths_to_prev = shortest_paths_to[prev_user["id"]]
        paths_via_prev = [path + [user_id] for path in paths_to_prev]

        # it's possible we already know a shortest path to here as well
        old_paths_to_here = shortest_paths_to.get(user_id, [])

        # what's the shortest path to here that we've seen so far?
        if old_paths_to_here:
            min_path_length = len(old_paths_to_here[0])
        else:
            min_path_length = float('inf')

        # any new paths to here that aren't too long
        new_paths_to_here = [path_via_prev
                             for path_via_prev in paths_via_prev
                             if len(path_via_prev) <= min_path_length
                             and path_via_prev not in old_paths_to_here]

        shortest_paths_to[user_id] = old_paths_to_here + new_paths_to_here

        # add new neighbors to the frontier
        frontier.extend((user, friend)
                        for friend in user["friends"]
                        if friend["id"] not in shortest_paths_to)

    return shortest_paths_to

Now we can store these `dicts` with each node:

In [4]:
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)

And we're finally ready to compute betweenness centrality.  For every pair of nodes i and j, we know the n shortest paths from i to j.  Then, for each of those paths, we just add 1/n to the centrality of each node on that path:

In [6]:
for user in users:
    user["betweenness_centrality"] = 0.0

for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].items():
        if source_id < target_id:   # don't double count
            num_paths = len(paths)  # how many shortest paths?
            contrib = 1 / num_paths # contribution to centrality
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib
                        
print("Betweenness Centrality")
for user in users:
    print(user["id"], user["betweenness_centrality"])
print()

Betweenness Centrality
0 0.0
1 3.5
2 3.5
3 18.0
4 20.0
5 20.5
6 6.0
7 6.0
8 8.5
9 0.0



Another measure we can look at is closeness centrality.  First, for each user we compute her farness, which is the sum of the lengths of her shortest paths to each other user.  Since we've already computed the shortest paths between each pair of nodes, it's easy to add their lengths:

In [8]:
def farness(user):
    """the sum of the lengths of the shortest paths to each other user"""
    return sum(len(paths[0])
               for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1 / farness(user)
    
print("Closeness Centrality")
for user in users:
    print(user["id"], user["closeness_centrality"])
print()

Closeness Centrality
0 0.029411764705882353
1 0.037037037037037035
2 0.037037037037037035
3 0.045454545454545456
4 0.05
5 0.05
6 0.041666666666666664
7 0.041666666666666664
8 0.03571428571428571
9 0.027777777777777776



There is much less variation here.  Even the very central nodes are still pretty far from the nodes out on the periphery.  For larger networks though, Eigenvector Centrality is used.

In order to talk aboout eigenvector centrality, we have to talk about eigenvectors.  And in order to talk about eigenvectors, we have to talk about matrix multiplication.  If A is a $ n_1 x k_1 $ and B is a $ n_2 x k_2 $ matrix, and if $ k_1 = n_2 $, then their produict AB is the $ n_1 x k_2 $ matrix whose (i,j)th entry is:

$$ A_ilB_kj + A_i2 B_2j + ... + A_ik B_kj $$

Which is just the `dot` product of the *i*th row of A (thought of as a vector) with the *j*th column of B (also thought of as a vector):

In [9]:
def matrix_product_entry(A, B, i, j):
    return dot(get_row(A, i), get_column(B, j))

After which we have:

In [10]:
def matrix_multiply(A, B):
    n1, k1 = shape(A)
    n2, k2 = shape(B)
    if k1 != n2:
        raise ArithmeticError("incompatible shapes!")

    return make_matrix(n1, k2, partial(matrix_product_entry, A, B))

Notice that if A is an $ n x k $ matrix and B is a $ k * 1 $ matrix, then AB is an $ n * 1 $ matrix.  If we treat a vector as a one-column matrix, we can think of A as a function that maps k-dimensional vectors to n-dimensional vectors, where the function is just matrix multiplication.

Previously, we represented  vectors simply as lists, which isn't quite the same, so we'll need to create some helper functions:

In [11]:
def vector_as_matrix(v):
    """returns the vector v (represented as a list) as a n x 1 matrix"""
    return [[v_i] for v_i in v]

def vector_from_matrix(v_as_matrix):
    """returns the n x 1 matrix as a list of values"""
    return [row[0] for row in v_as_matrix]

When A is a square matrix, this operation maps n-dimensional vectors to other n-diumensional vectors.  It's possible that for some matrix A and vector v, when A operates on v we get back a scalar multiple of v.  That is, that the result is a vector that points in the same direction as v.  When this happens (and when, in addition, v is not a vector of all zeroes), we call v an eigenvector of A.  And we call the multiplier an eigenvalue.

One possible way to find an eigenvector of A is b y picking a starting vector v, applying `matrix_operate`, rescaling the result to have magnitude 1, and repeating until the process converges:

In [12]:
def matrix_operate(A, v):
    v_as_matrix = vector_as_matrix(v)
    product = matrix_multiply(A, v_as_matrix)
    return vector_from_matrix(product)

def find_eigenvector(A, tolerance=0.00001):
    guess = [1 for __ in A]

    while True:
        result = matrix_operate(A, guess)
        length = magnitude(result)
        next_guess = scalar_multiply(1/length, result)

        if distance(guess, next_guess) < tolerance:
            return next_guess, length # eigenvector, eigenvalue

        guess = next_guess

By construction, the returned `guess` is a vector such that, when you apply `matrix_operate` to it and rescale it to have length 1, you get back itself.  Which means it's an eigenvector.

Not all matrices of real numbers have eigenvectors and eigenvalues.  for example the matrix:

```python
rotate = [[ 0, 1],
          [ -1, 0]]
```

rotates vectors 90 degrees clockwise, which means that the only vector it maps to a scalar multple of itself is a vector of zeroes.  If you tried `find_eigenvector(rotate)` it would run forever.  Even matrices that have eigenvectors can sometimes get stuck in cycles.  Consider the matrix:

```python
flip = [[ 0, 1],
        [1, 0]]
```

This matrix maps any vector to its inverse.  It will run forever, but when it does stop it will return an eigenvector.  So how does this help us understand our network?  To start with, we'll need to represent the connections in our network as an `adjacency_matrix`, whose (i,j)th entry is either 1 or 0:

In [16]:
def entry_fn(i, j):
    return 1 if (i, j) in friendships or (j, i) in friendships else 0

n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

print("Eigenvector Centrality")
for user_id, centrality in enumerate(eigenvector_centralities):
    print(user_id, centrality)
print()

Eigenvector Centrality
0 0.38578006614957344
1 0.5147902322356226
2 0.5147902322356226
3 0.47331220396377677
4 0.23361029944966002
5 0.1501458150031844
6 0.08355561051056493
7 0.08355561051056493
8 0.07284034177922594
9 0.027294660139652423



The eigenvector centrality for each user is then the entry corresponding to that user in the eigenvector returned  by `find_eigenvector`.  Users with high eigenvector centrality should be those who have a lot of connections and connections to people who themselves have high centrality.  Here users 1 and 2 are the most central, as they both have three conections to people who are themselves highly central. As we move away from them, people's centralities steadily drop off.

On a network this small, eigenvector centrality behaves somewhat erratically.  If you tryn adding or subtracting links, you'll find that small changes in the network can dramatically change the centrality numbers.  In a much larger network this would not particularly be the case.

We still haven't motivated why an eigenvector might lead to a reasonable notion of centrality.  Being an eigenvector means that if you compute:

```python
matrix_operate(adjacency_matrix, eigenvector_centralities)
```

this results in a scalar multiple of `eigenvector_centralities`.  If you look at how matrix multiplication works, `matrix_operate` produces a vector whose *i*th element is:

```python
dot(get_row(adjacency_matrix, i), eigenvector_centralities)
```

Which is precisely the sum of the eigenvector centralities of the users connected to user *i*.  In other words, eigenvector centralities are numbers, one per user, such that each user's value is a constant multiple of the sum of his neighbors' values.  In this case, centrality means being connected to people who themselves are central.  The more centrality you are directly connected to, the more central you are.  This is of course a circular definition; eigenvectors are the way of breaking out of the circularity.

Another way of understanding this is by thinking about what `find_eigenvector` is doing here.  It starts by assigning each node a random centrality.  It them repeats the following two steps until the process converges:

1. Give each node a new centrality score that equals the sum of its neighbors' old centrality scores.
2. Rescale the vector of centralities to have magnitude 1.

Although the mathematics behind it may seem somewhat opaque at first, the calculation itse;f is relatively straightforward and is pretty easy to perform on even very large graphs.

We'll track endorsements `(source, target)` that no longer represent a reciprocal relationship, but rather that source endorses `target` as an awesome data scientist.  We'll need to account for this assemetry:

In [19]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1), (1, 3),
                (2, 3), (3, 4), (5, 4), (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # add one list to track outgoing endorsements
    user["endorsed_by"] = []    # and another to track endorsements

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])

After which we can easily find the `most_endorsed` data scientists and sell that information to recruiters:

In [20]:
endorsements_by_id = [(user["id"], len(user["endorsed_by"]))
                      for user in users]

sorted(endorsements_by_id,
       key=lambda pair: pair[1],
       reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

However, "number of endorsements" is an easy metric to game.  All you need to do is create phony accounts and have them endorse you.  Or arrange with your friends to endorse each other.  A better metric would take into account *who* endorses you.  Endorsements from people who have a lot of endorsements should somehow count more than endorsements from people with few endorsements.  This is the essence of the PageRank algorithm, used by Google to rank websites based on which other websites link to them, which other websites link to those, and so on.

A simplified version looks like:

1. There is a total of 1.0 (or 100%) PageRank in the network.
2. Initially this PageRank is equally distributed among nodes.
3. At each step, a large fraction of each node's PageRank is distributed evenly among jits outgoing links.
4. At each step, the remainder of each node's PageRank is distributed evenly among all nodes.

Here's what that code looks like:

In [21]:
def page_rank(users, damping = 0.85, num_iters = 100):

    # initially distribute PageRank evenly
    num_users = len(users)
    pr = { user["id"] : 1 / num_users for user in users }

    # this is the small fraction of PageRank
    # that each node gets each iteration
    base_pr = (1 - damping) / num_users

    for __ in range(num_iters):
        next_pr = { user["id"] : base_pr for user in users }
        for user in users:
            # distribute PageRank to outgoing links
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += links_pr / len(user["endorses"])

        pr = next_pr

    return pr

print("PageRank")
for user_id, pr in page_rank(users).items():
    print(user_id, pr)

PageRank
0 0.0404553415061296
1 0.044921190893169885
2 0.044921190893169885
3 0.0404553415061296
4 0.06785083675770529
5 0.04344422700587085
6 0.03346379647749512
7 0.03346379647749512
8 0.04344422700587085
9 0.03346379647749512


Even though he has few endorsements than users 0, 1, and 2, his endorsements carry with them rank from their endorsements.  Additionally, both of his endorsers endorsed only him, which means that he doesn't have to divide their rank with anyone else.